In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import unicodedata
import re

## Join together UW orgs with the CRA/211 data

Find missing UW orgs from 2025

In [42]:
# Clean text functions
def clean_text(text):
    if pd.isna(text):
        return ''
    
    text = unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'/.*$', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\b(the|inc|incorporated|llc|corp|corporation|co|company)\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z0-9]', '', text).lower()
    
    return text.strip()

def clean_text_with_spaces(text):
    if pd.isna(text):
        return ''
    
    text = unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'/.*$', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\b(the|inc|incorporated|llc|corp|corporation|co|company)\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text).lower()
    
    text = re.sub(r' +', ' ', text).strip()
    
    return text

# Load the dataframes
# df_uw_cra211 = pd.read_csv('../joined-data/uw_cra211.csv', encoding='latin')
df_uw_cra211 = pd.read_excel('../uw-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw3 = pd.read_csv('../uw-data/2024-25 Program Locations for Mapping(Sheet2).csv', encoding='latin')

# Create dummy columns for comparison (without spaces)
df_uw_cra211['dummy'] = df_uw_cra211['Funded Agency - CRA Name'].apply(clean_text)
df_uw3['dummy'] = df_uw3['Organization Name: | Legal Name'].apply(clean_text)
df_uw3['dummy_space'] = df_uw3['Organization Name: | Legal Name'].apply(clean_text_with_spaces)

# Get unique organization names from df_uw3
uw3_orgs = df_uw3['Organization Name: | Legal Name'].unique()
uw3_dummies = df_uw3['dummy'].unique()
uw3_dummies_space = df_uw3['dummy_space'].unique()

# Find which organizations don't have matches in df_uw_cra211
missing_orgs = []
for dummy_space, dummy in zip(uw3_dummies_space, uw3_dummies):
    if dummy not in df_uw_cra211['dummy'].values:
        # Get the clean version with spaces for reporting
        missing_orgs.append(dummy_space)

# Report the results
# if len(missing_orgs) > 0:
#     print("Organizations in df_uw3 that don't have matches in df_uw_cra211:")
#     for i, org in enumerate(missing_orgs, 1):
#         print(f"{i}. {org}")
# else:
#     print("All organizations in df_uw3 have matches in df_uw_cra211")

df_uw_missing = df_uw3[df_uw3['dummy_space'].isin(missing_orgs)][['Organization Name: | Legal Name', 'Organization Name: | Operating/Public Name']].drop_duplicates()
df_uw_missing.to_csv('../uw-data/2025-uw-missing.csv', index=False)

### Join together all UW data

In [2]:
df_matched_orgs = pd.read_csv('../joined-data/simplified_matches.csv')
df_matched_orgs = df_matched_orgs.drop(columns=['CRA_Category', 'CRA_SubCategory'])

df_uw = pd.read_excel('../uw-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw = df_uw.rename(columns={'CHARITABLE REGISTRATION NUMBER (BN)': 'CRA_BN_ID'})

In [3]:
df_manual_uw_map = pd.read_csv('../uw-data/manual_matches_uw.csv')

# Load 211 data, select columns only relevant to simplified form, and rename them
df_211 = pd.read_csv("../211-data/2021_211_PeelYorkTO.csv", encoding='latin-1')
df_211 = df_211.drop(columns=["TaxonomyTerms", "Address2", "County", "Province"])
df_211 = df_211.rename(columns={
    "PublicName": "211_PublicName",
    "ParentAgency": "211_Name",
    "Address1":"211_Address1",
    "City":"211_City",
    "PostalCode":"211_PostalCode",
    "Latitude": "211_Latitude",
    "Longitude": "211_Longitude",
}).rename(columns={
    '211_Name': '211_Organization_Name',
    '211_PublicName': '211_Location_Name',
    '211_Address1': '211_Address',
    '211_City': '211_City',
    '211_PostalCode': '211_Postal_Code',
    '211_Longitude': 'X_Coordinate',
    '211_Latitude': 'Y_Coordinate'
})
df_211 = df_211.apply(lambda x: x.str.replace(';', ' - ') if x.dtype == 'object' else x)

Join together manual matches 

In [5]:
df_manual_orgs = pd.merge(
    left = df_manual_uw_map,
    right = df_211,
    on = '211_Organization_Name',
    how = 'left'
)
df_manual_orgs['Match_Method'] = 'Manual UW'

In [11]:
df_uw_cra211_manual = pd.merge(
    left = df_uw,
    right = df_manual_orgs,
    on = 'Funded Agency - CRA Name',
    how = 'inner'  # Identify only these manual matches
)

For other organizations, use the CRA ID

In [13]:
df_uw2 = df_uw[~df_uw['Funded Agency - CRA Name'].isin(df_uw_cra211_manual['Funded Agency - CRA Name'])]
df_uw_cra211_bnid = pd.merge(
    left = df_uw2,
    right = df_matched_orgs,
    on = 'CRA_BN_ID',
    how = 'left'
)

Incorporate the August 2025 version of UW organizations (REDUNDANT)

In [43]:
# df_uw25_map = pd.read_csv('../uw-data/2025-uw-missing-matches.csv').dropna().drop(columns=['UW 2025'])
# df_uw25_map = df_uw25_map[~df_uw25_map['Funded Agency - CRA Name'].isin(df_uw['Funded Agency - CRA Name'])]
# df_uw25_map[['CRA_BN_ID', 'Category', 'Sub Category', 'City', 'Reason', 'Region']] = np.nan

# df_uw25_manual = pd.merge(
#     left = df_uw25_map,
#     right = df_211,
#     on = '211_Organization_Name',
#     how = 'left'
# )
# df_uw25_manual['Match_Method'] = 'Manual UW 25'

Concat

In [44]:
df_uw_cra211 = pd.concat([df_uw_cra211_manual, df_uw_cra211_bnid])
df_uw_cra211 = df_uw_cra211.sort_values(by='CRA_Organization_Name')
df_uw_cra211.to_csv('../joined-data/uw_cra211.csv', index=False)

In [19]:
df_uw_cra211.columns

Index(['Funded Agency - CRA Name', 'CRA_BN_ID', 'Category', 'Sub Category',
       'City', 'Reason', 'Region', 'CRA_Organization_Name',
       '211_Organization_Name', '211_Location_Name', '211_Address', '211_City',
       '211_Postal_Code', 'Y_Coordinate', 'X_Coordinate', 'Match_Method'],
      dtype='object')

In [18]:
print(f'Total UW orgs: {len(df_uw)}')

num_uw_matches = df_uw_cra211['CRA_Organization_Name'].nunique()
print(f'No. UW orgs with 211/CRA data: {num_uw_matches}')

Total UW orgs: 278
No. UW orgs with 211/CRA data: 226


## Join tenure data

In [9]:
df_tenure_by_cra = pd.read_excel('../../2024-work/SPRE_data/2021_CRE_DATA_Oct9.xlsx')
df_tenure_by_cra = df_tenure_by_cra.drop_duplicates()
# ['CRA_BN', 'CRA_LegalName', '211 Parent Agency Name', '211_Address1', '211_PostalCode', 'Tenure', 'Latitude', 'Longitude']
df_tenure_by_cra = df_tenure_by_cra[['CRA_BN', 'Latitude', 'Longitude', 'Tenure']].rename(
    columns={
        'CRA_BN': 'CRA_BN_ID',
        # 'CRA_LegalName': 'CRA_Organization_Name',
        # '211 Parent Agency Name': '211_Organization_Name',
        # '211_Address1': '211_Address',
        # '211_PostalCode': '211_Postal_Code',
        'Latitude': 'Y_Coordinate',
        'Longitude': 'X_Coordinate',
    }
)

# df_tenure_by_cra

In [10]:
df_uw_cra211_tenure = pd.merge(
    left = df_uw_cra211,
    right = df_tenure_by_cra,
    on = ['CRA_BN_ID', 'Y_Coordinate', 'X_Coordinate'],
    how = 'left'
)
df_uw_cra211_tenure.to_csv('../joined-data/uw_cra211_tenure.csv', index=False)

num_tenure = df_uw_cra211_tenure['Tenure'].count()
num_locs = len(df_uw_cra211_tenure)
pct_tenure = round(num_tenure / num_locs, 4) * 100
print(f'Number of locations with tenure data: {num_tenure}')
print(f'...which is {pct_tenure}% of all locations')
print(f'...so {num_locs - num_tenure} locations are missing tenure data')
print(f'...as there are {num_locs} locations in total')

Number of locations with tenure data: 430
...which is 43.57% of all locations
...so 557 locations are missing tenure data
...as there are 987 locations in total


## Join tenure data to simplified_matches and extend it

In [11]:
df_matched_orgs = pd.read_csv('../joined-data/simplified_matches.csv')

# Add the UW-only CRA organizations
df_matched_extended = pd.concat([
    df_matched_orgs,
    df_uw_cra211_manual.drop(columns=['Funded Agency - CRA Name', 'City', 'Reason', 'Region']).rename(columns={'Category': 'CRA_Category', 'Sub Category': 'CRA_SubCategory'})
]).sort_values(by='CRA_Organization_Name')

# Add tenure information
df_matched_extended = pd.merge(
    left = df_matched_extended,
    right = df_tenure_by_cra,
    on = ['CRA_BN_ID', 'Y_Coordinate', 'X_Coordinate'],
    how = 'left'
)

# Specify with an organization is UW or not
df_matched_extended['UW'] = df_matched_extended['CRA_Organization_Name'].isin(df_uw_cra211_tenure['CRA_Organization_Name'])

df_matched_extended.to_csv('../joined-data/simplified_matches_uw_tenure.csv', index=False)

num_tenure = df_matched_extended['Tenure'].count()
num_locs = len(df_matched_extended)
pct_tenure = round(num_tenure / num_locs, 4) * 100
print(f'Number of all locations with tenure data: {num_tenure}')
print(f'...which is {pct_tenure}% of all locations')
print(f'...so {num_locs - num_tenure} locations are missing tenure data')
print(f'...as there are {num_locs} locations in total')

Number of all locations with tenure data: 1042
...which is 46.29% of all locations
...so 1209 locations are missing tenure data
...as there are 2251 locations in total


In [23]:
df_matched_extended = pd.read_csv('../joined-data/simplified_matches_uw_tenure.csv')

# Filter to only rows with no Tenure
df_matched_extended_nt = df_matched_extended[df_matched_extended['Tenure'].isna()]

# Count occurrences
name_counts = df_matched_extended_nt['211_Organization_Name'].value_counts()

# Split into duplicates and uniques
df_dupes = df_matched_extended_nt[df_matched_extended_nt['211_Organization_Name'].isin(name_counts[name_counts > 1].index)]
df_uniques = df_matched_extended_nt[df_matched_extended_nt['211_Organization_Name'].isin(name_counts[name_counts == 1].index)]

# Save the version with only repeated orgs
df_dupes.sort_values(by='211_City').to_csv('../joined-data/simplified_matches_uw_no_tenure.csv', index=False)

# Save the version with only unique orgs (if needed)
df_uniques.sort_values(by='211_City').to_csv('../joined-data/simplified_matches_uw_no_tenure_uniques_only.csv', index=False)

# Optionally, view counts
print(df_dupes['211_City'].value_counts())

211_City
Toronto                   772
Mississauga                45
Brampton                   37
Newmarket                  25
Markham                    24
Richmond Hill              24
Vaughan                    17
Aurora                     17
Georgina                    5
Whitchurch-Stouffville      2
King                        2
East Gwillimbury            2
Caledon                     1
Name: count, dtype: int64


## Join manually retrieved tenure data

Merge in Toronto data

In [2]:
df_matches_all = pd.read_csv('../joined-data/simplified_matches_uw_tenure.csv')
df_tenure_to = pd.read_csv('../tenure-data/Non-Profit UW Tenure Toronto - Sheet1.csv')

In [3]:
merge_cols = ['CRA_Organization_Name', '211_Organization_Name', '211_Location_Name', '211_Address', '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate']

# Merge tenure info from df_tenure_to into df_matches_all
df_merged = pd.merge(
    df_matches_all,
    df_tenure_to[merge_cols + ['Tenure (Rent or Own or Unknown)']],
    on=merge_cols,
    how='left'
)

# Create new column 'Tenure_Public' initialized as empty string
df_merged['Tenure_Public'] = ''

# Function to update tenure values
def update_tenure(row):
    current_tenure = row['Tenure'] 
    new_tenure_info = row['Tenure (Rent or Own or Unknown)']
    
    # Only update if current tenure is 'Unknown' or 'Empty'
    if current_tenure == 'Unknown' or pd.isna(current_tenure):
        if pd.isna(new_tenure_info):
            return row['Tenure'], ''
        elif str(new_tenure_info).startswith('City of '):
            return 'Rent', new_tenure_info
        else:
            return new_tenure_info, ''
    else:
        return row['Tenure'], ''

# Apply function to update Tenure and Tenure_Public columns
df_merged[['Tenure', 'Tenure_Public']] = df_merged.apply(update_tenure, axis=1, result_type='expand')

# Drop helper column from df_tenure_to
df_matches_all_updated = df_merged.drop(columns=['Tenure (Rent or Own or Unknown)'])

Merge in non-Toronto GTA data

In [4]:
df_tenure_gta = pd.read_csv('../tenure-data/2021_Joinedv2_short - Copy(CRE2021).csv')

In [5]:
# Ensure the column is named 'Tenure_GTA' (skip renaming again if already done)
# If it’s not renamed yet, you can uncomment the next line:
df_tenure_gta = df_tenure_gta.rename(columns={'Tenure': 'Tenure_GTA'})

# Clean up df_tenure_gta['Tenure_GTA']: replace non 'Rent', 'Own', 'Unknown' with 'Unknown'
df_tenure_gta['Tenure_GTA'] = df_tenure_gta['Tenure_GTA'].str.strip()
df_tenure_gta['Tenure_GTA'] = df_tenure_gta['Tenure_GTA'].where(df_tenure_gta['Tenure_GTA'].isin(['Rent', 'Own', 'Unknown']), 'Unknown')

merge_cols = ['CRA_Organization_Name', '211_Organization_Name', '211_Location_Name', '211_Address', '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate']

# Group by merge columns and resolve conflicts
def resolve_tenure(series):
    unique_tenures = series.dropna().unique()
    if len(unique_tenures) == 1:
        return unique_tenures[0]
    else:
        return 'Unknown'

df_tenure_gta_agg = df_tenure_gta.groupby(merge_cols)['Tenure_GTA'].apply(resolve_tenure).reset_index()

# Merge the resolved GTA tenure info into df_matches_all_updated
df_merged_gta = pd.merge(
    df_matches_all_updated,
    df_tenure_gta_agg,
    on=merge_cols,
    how='left'
)

# Update 'Tenure' where it is 'Unknown' or NaN, using 'Tenure_GTA' if available
def update_tenure_gta(row):
    current_tenure = str(row['Tenure']).strip().lower()
    gta_tenure = row['Tenure_GTA']
    
    if current_tenure == 'unknown' or pd.isna(row['Tenure']):
        if pd.notna(gta_tenure) and gta_tenure != 'Unknown':
            return gta_tenure
    return row['Tenure']

df_merged_gta['Tenure'] = df_merged_gta.apply(update_tenure_gta, axis=1)

# Drop helper column
df_matches_all_final = df_merged_gta.drop(columns=['Tenure_GTA'])

Set single property values to 'Own' if we detect a property value

In [6]:
df_matches_all_final.columns

Index(['CRA_BN_ID', 'CRA_Organization_Name', 'CRA_Category', 'CRA_SubCategory',
       '211_Organization_Name', '211_Location_Name', '211_Address', '211_City',
       '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate', 'Match_Method',
       'Tenure', 'UW', 'Tenure_Public'],
      dtype='object')

In [7]:
df_land_status = pd.read_csv('../tenure-data/2021_Joinedv2(CRE2021).csv', encoding='latin')
df_land_status = df_land_status[[
    'CRA_BN_ID', 'CRA_Organi', 'F211_Organ', 'F211_Locat', 'F211_Addre',
    'F211_Posta', 'X_Coordina', 'Y_Coordina', 'Match_Meth', 'Tenure', 'UW', 
    'Land_or_Bldg_owned_4050', 'land_bldg_4155',
]].rename(columns={
    'CRA_Organi': 'CRA_Organization_Name',
    'F211_Organ': '211_Organization_Name',
    'F211_Locat': '211_Location_Name', 
    'F211_Addre': '211_Address',
    'F211_Posta': '211_Postal_Code',
    'X_Coordina': 'X_Coordinate',
    'Y_Coordina': 'Y_Coordinate',
    'Match_Meth': 'Match_Method',
})
df_land_status.columns

Index(['CRA_BN_ID', 'CRA_Organization_Name', '211_Organization_Name',
       '211_Location_Name', '211_Address', '211_Postal_Code', 'X_Coordinate',
       'Y_Coordinate', 'Match_Method', 'Tenure', 'UW',
       'Land_or_Bldg_owned_4050', 'land_bldg_4155'],
      dtype='object')

In [8]:
# Step 1: Clean up whitespace in relevant columns
df_land_status['Land_or_Bldg_owned_4050'] = df_land_status['Land_or_Bldg_owned_4050'].str.strip()
df_land_status['land_bldg_4155'] = df_land_status['land_bldg_4155'].str.strip()

# Step 2: Convert 'land_bldg_4155' to integer (handle NaNs or empty strings)
df_land_status['land_bldg_4155'] = pd.to_numeric(df_land_status['land_bldg_4155'], errors='coerce').fillna(0).astype(int)

# Step 3: Keep only rows where '211_Organization_Name' is unique
unique_orgs = df_land_status['211_Organization_Name'].value_counts()
unique_org_names = unique_orgs[unique_orgs == 1].index
df_land_status_unique = df_land_status[df_land_status['211_Organization_Name'].isin(unique_org_names)].copy()

# Step 4: Update 'Tenure' based on 'Land_or_Bldg_owned_4050' and 'land_bldg_4155'
def infer_tenure(row):
    current_tenure = row['Tenure']
    owned_status = row['Land_or_Bldg_owned_4050']
    building_value = row['land_bldg_4155']
    
    if current_tenure in ['Rent', 'Own']:
        return current_tenure  # Leave as is
    if pd.notna(owned_status) and owned_status != 'N' and building_value > 0:
        return 'Own'
    else:
        return 'Rent'

df_land_status_unique['Tenure_LandStatus'] = df_land_status_unique.apply(infer_tenure, axis=1)

# Step 5: Prepare for merge by keeping necessary columns
merge_cols = ['CRA_Organization_Name', '211_Organization_Name', '211_Location_Name', '211_Address', '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate']
df_land_status_merge = df_land_status_unique[merge_cols + ['Tenure_LandStatus']]

# Step 6: Merge into df_matches_all_final
df_merged_land = pd.merge(
    df_matches_all_final,
    df_land_status_merge,
    on=merge_cols,
    how='left'
)

# Step 7: Update 'Tenure' where it's 'Unknown' or NaN, using 'Tenure_LandStatus'
def update_tenure_land(row):
    current_tenure = str(row['Tenure']).strip().lower()
    land_tenure = row['Tenure_LandStatus']
    
    if current_tenure == 'unknown' or pd.isna(row['Tenure']):
        if pd.notna(land_tenure):
            return land_tenure
    return row['Tenure']

df_merged_land['Tenure'] = df_merged_land.apply(update_tenure_land, axis=1)

# Step 8: Drop helper column
df_matches_all_final_updated = df_merged_land.drop(columns=['Tenure_LandStatus'])

In [9]:
df_matches_all_final_updated['Tenure'].value_counts()

Tenure
Rent       1326
Own         545
Unknown     173
Name: count, dtype: int64

Join Sarah's manual listing of tenure

In [26]:
df_sarah = pd.read_csv('../tenure-data/simplified_matches_uw_tenure_SCupdate(in).csv')

In [22]:
# Step 1: Define merge columns
merge_cols = [
    'CRA_Organization_Name', '211_Organization_Name', '211_Location_Name',
    '211_Address', '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate'
]

# Step 2: Prepare df_sarah for merge (keep only necessary columns)
df_sarah_merge = df_sarah[merge_cols + ['Tenure']].rename(columns={'Tenure': 'Tenure_Sarah'})

# Step 3: Merge with df_matches_all_final_updated
df_merged_sarah = pd.merge(
    df_matches_all_final_updated,
    df_sarah_merge,
    on=merge_cols,
    how='left'
)

# Step 4: Update Tenure only if current is not 'Rent' or 'Own'
def update_tenure_sarah(row):
    current_tenure = str(row['Tenure']).strip().lower()
    sarah_tenure = row['Tenure_Sarah']
    
    if current_tenure not in ['rent', 'own'] and pd.notna(sarah_tenure):
        return sarah_tenure
    return row['Tenure']

df_merged_sarah['Tenure'] = df_merged_sarah.apply(update_tenure_sarah, axis=1)

# Step 5: Drop helper column
df_matches_all_final_sc = df_merged_sarah.drop(columns=['Tenure_Sarah'])

In [23]:
df_matches_all_final_sc['Tenure'].value_counts()

Tenure
Rent       1477
Own         562
Unknown     174
Name: count, dtype: int64

In [25]:
df_matches_all_final_sc.to_csv('../joined-data/simplified_matches_uw_tenure_final.csv', index=False)

## Save data as GeoJSON with updated columns

In [27]:
df_matches_all = pd.read_csv('../joined-data/simplified_matches_uw_tenure_final.csv')
df_matches_all.columns

Index(['CRA_BN_ID', 'CRA_Organization_Name', 'CRA_Category', 'CRA_SubCategory',
       '211_Organization_Name', '211_Location_Name', '211_Address', '211_City',
       '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate', 'Match_Method',
       'Tenure', 'UW', 'Tenure_Public'],
      dtype='object')

In [28]:
df_matches_all['Tenure'] = df_matches_all['Tenure'].fillna('Unknown')
df_matches_all = df_matches_all.drop(columns=['CRA_BN_ID', 'CRA_Organization_Name', '211_Location_Name', '211_Postal_Code', 'Match_Method', 'UW', 'Tenure_Public'])
df_matches_all = df_matches_all.rename(columns={
    '211_Organization_Name': 'N',
    '211_Address': 'A',
    'CRA_Category': 'C',
    'CRA_SubCategory': 'S',
    'Tenure': 'T',
})

In [29]:
gdf_matches = gpd.GeoDataFrame(
    df_matches_all, geometry=gpd.points_from_xy(df_matches_all.X_Coordinate, df_matches_all.Y_Coordinate), crs="EPSG:4326"
)
gdf_matches.to_file('../joined-data/simplified_matches_4326.geo.json', driver='GeoJSON')